<a href="https://colab.research.google.com/github/inoyamanaka/Pemrosesan_Text/blob/main/Tf-IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tweepy
import csv
import re
from google.colab import files
!pip install git+https://github.com/tweepy/tweepy.git

In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

pd.set_option('display.max_rows', None)

In [ ]:
api_key = "wjfC7sqEsgkkOtAh3QUDKTTGJ"
api_secret = "TAwEmJ1MhYMgKwJBngh4ZpXnWdNFbdj778fiq6UkLFuLZ5ExKQ"
access_token = "1198104989102567424-noL01z6l2PvZvtfgwhrebESA3dmwN9"
access_secret = "gFTvVQoIO4J7gZ2yqVf5It6GaZmbtCz61FQphSfVJIHTJ"

auth = tweepy.OAuthHandler(api_key,api_secret)
auth.set_access_token(access_token,access_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)
api.verify_credentials()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## PROSES CRAWLING DATA DARI TWITTER

In [19]:
output = []
for tweet in tweepy.Cursor(api.search_tweets, "Kkb Papua", count=50).items(50):
    user = tweet.user.screen_name
    tweet_id = tweet.id
    list_output = (tweet.text)
    output.append(list_output)

In [ ]:
outputs = pd.read_csv('output (10).csv', names=['Text'])
outputs

In [ ]:
df = pd.DataFrame(outputs,columns =['Text'])
df.head(20)

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## PROSES CASE FOLDING

In [22]:
def preprocess(str_text):
  # 1. Casefolding - mengubah ke case yang sama semua (misal huruf kecil semua)
  str_text = str_text.lower()
  return str_text

df['Hasil Case Folding'] = df['Text'].apply(lambda x: preprocess(x))

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## PROSES TOKENIZE

In [ ]:
def preprocess(str_text):
  # 1. Hapus mention (@..) - menggunakan regular expression
  str_text = re.sub(r'@[\w]+',' ',str_text)
  # 2. Hapus hashtag (#..) - menggunakan regular expression
  str_text = re.sub(r'#[\w]+',' ',str_text)
  # Langkah 2 dan 3 bisa digabung dengan kode re: r'[@|#][\w]+'
  # 3. Hapus URL
  str_text = re.sub(r'http\S+',' ',str_text)
  # 4. Hapus non-printable chars, tanda baca, angka (alias hapus selain huruf)
  str_text = re.sub(r'[^a-zA-Z]+',' ',str_text)

  # Tokenize kalimat
  str_text = re.split("\s", str_text)
  return str_text

df['Hasil Tokenize'] = df['Hasil Case Folding'].apply(lambda x: preprocess(x))
df

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## PROSES FILTERING KATA

In [ ]:
# Filtering words
ban_words = ['rt','dan', 'dari' ,'serta' ,'melainkan' ,'padahal' ,'sedangkan' ,'atau', 'tetapi',
             'jika','jikalau','bahwa','sehingga','sudah', 'yang','padahal', 'sedangkan', 'sambil',
             'supaya', 'agar', 'untuk','yg','aja','ini','daripada','ke','di','n','dengan','jadi',
             'd','adanya','saat','bahkan','sangat','dah','gw','doang','aja','ini','hanya','dalam',
             'tak','adalah','adanya','termasu','karena','mengapa', 'tidak', 'ada','b','t','g','p',
              'hrus', 'bisa','dob','bukan','dalam','malah','akibat','pem','md','lagi','tapi','sono',
             'sana','drun','bacot', 'p','pua','terbaikdi','drun','gorok','oap','pua', 'mbak', 'kampung',
             'tumb', 'lkan','ny','usah','ntai','roris','ring','artinya','juga','ara','dong','yaa','cuma',
             'gore','akan','maybrat','oleh','sugapa','k','hari','akh','mengut','sebenarnya','pa',
             'begitu','kira','kurang','lebihnya','bro','jelas','malah','aneh','anies','saatnya',
             'terima','kasih','ammi','gat','hendrik','']


def filter2(str_text):
  # Hapus kata ban words
  for i in str_text:
    if i in ban_words:
      str_text.remove(i)

  return str_text
    
# df['Hasil Filtering'] = df['Hasil Tokenize'].apply(lambda x: filter1(x))
df['Hasil Filtering'] = df['Hasil Tokenize'].apply(lambda x: filter2(x))
df[['Hasil Filtering']]

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## PROSES STEMMING

In [25]:
# STEMMING
# kamus normalisasi
norm = {'pembentukan':'bentuk','pendidikan':'didik','bukanlah':'bukan','pedalaman':'dalam','terdepan':'depan',
        'kesejahteraan':'sejahtera','terwujud':'wujud','pemekaran':'mekar','dianggap':'anggap','lihatlah':'lihat',
        'dihentikan':'henti','menghambat':'hambat','pembangunan':'bangun','berdemo':'demo','menolak':'tolak',
        'pendukung':'dukung','menebarkan':'tebar','mengumumkan':'umum','nguasain':'kuasa','terbaik':'baik','pekerja':'kerja',
        'tersangka':'sangka','merajalela':'rajalela','melakukan':'laku','menghambat':'hambat','kemajuan':'maju','memasok':'pasok',
        'mengungsi':'ungsi','penyerangan':'serang','mengakibatkan':'akibat','ditembak':'tembak','berhari':'hari','penanganan':'angan',
        'menggangap':'anggap','menghalangi':'alang','tersangka':'sangka','terduga':'duga','membangun':'bangun','menganggap':'anggap',
        'mendukung':'dukung','memajukan':'maju','serangan':'serang','pendekatan':'dekat'
        }

In [ ]:
def preprocess(str_text):
  replacer = norm.get  
  return [replacer(n, n) for n in str_text]

df['Hasil Stemming'] = df['Hasil Filtering'].apply(lambda x: preprocess(x))
df[['Hasil Stemming']]

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## PROSES MENYIMPAN KEDALAM EXCEL

In [27]:
df[['Text','Hasil Case Folding']].to_excel("compact.xlsx")

In [ ]:
df[['Hasil Case Folding','Hasil Tokenize']].to_excel("compact1.xlsx")
df[['Hasil Tokenize','Hasil Filtering']].to_excel("compact2.xlsx")
df[['Hasil Filtering','Hasil Stemming']].to_excel("compact3.xlsx")

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## PROSES TF-IDF SCRATCH

In [28]:
Mylist = []
def gabung(str_text):
  Mylist.extend(str_text)
  return Mylist

def my_function(x):
  return list(dict.fromkeys(x))
    
baru = df['Hasil Stemming'].apply(lambda x: gabung(x))
data_baru = baru[0]

In [31]:
data_baru = my_function(data_baru)
data_baru.remove('')
word_set = data_baru

In [ ]:
word_set

In [ ]:
# MENGHITUNG JUMLAH KATA YANG KEMBAR PADA MASING-MASING TEXT
def cnt(str_text):
  word_count = {}
  jumlah = []
  for word in (word_set):
      word_count[word] = 0
      for sent in str_text:
          if word in sent:
              word_count[word] += 1

  # MENGEBALIKAN JUMLAH KATA DALAM BENTUK DICRIONARY
  for key in word_count:
    jumlah.append(word_count[key])
  return jumlah


# MEMBUAT KOLOM STEMMING MENJADI DALAM BENTUK LIST
def gabung2(str_text):
  Mylist2 = []
  Mylist2.extend(str_text)
  return Mylist2
    
# MEMBUAT TABEL BARU DENGAN ATRIBUT KATA
df_count = pd.DataFrame(word_set, columns =['Kata'])

# MEMANGGIL FUNGSI GABUNG2 
we_n = df['Hasil Stemming'].apply(lambda x: gabung2(x))

# MELAKUKAN LOOPING SEMBARI MENAMBAHKAN KOLOM PADA TABEL
for i in range (1,51):
  df_count[f'D{i}'] = cnt(we_n[i-1])

df_count


In [34]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
#TF-IDF Vectorize


In [ ]:
df_count.to_excel("final.xlsx")